In [43]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [44]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv('hitungan_manual.csv')

# Display the DataFrame
data = df[['full_text','sentimen']]
data

,full_text,sentimen
0,Dukungan trus mengalir untuk prabowo gibran se...,positif
1,Tetap kita dukung prabowo gibran,positif
2,Kita mendukung untuk Prabowo Gibran,positif
3,Ngeri tkut main Curang kaya 2019,negatif
4,Lah sekarang juga udah keliatan curangnya,negatif
5,Klo dipaksa satu putaran dengan kecurangan ya ...,negatif


In [45]:
# Mengubah kata Huruf besar menjadi huruf kecil
data['full_text'] = data['full_text'].str.lower()
data

,full_text,sentimen
0,dukungan trus mengalir untuk prabowo gibran se...,positif
1,tetap kita dukung prabowo gibran,positif
2,kita mendukung untuk prabowo gibran,positif
3,ngeri tkut main curang kaya 2019,negatif
4,lah sekarang juga udah keliatan curangnya,negatif
5,klo dipaksa satu putaran dengan kecurangan ya ...,negatif


In [46]:
# Menghilangkan kata-kata yang tidak memilki bobot
import Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
more_stop_word = ["tidak","prabowo","gibran","anis","muhaimin","ganjar","mahfud"]

stop_words = StopWordRemoverFactory().get_stop_words()
stop_words.extend(more_stop_word)

new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)

def stopword(str_text):
    str_text = stop_words_remover_new.remove(str_text)
    return str_text

data['full_text'] = data['full_text'].apply(lambda x: stopword(x))
data

,full_text,sentimen
0,dukungan trus mengalir prabowo semakin kuat,positif
1,tetap dukung gibran,positif
2,mendukung prabowo,positif
3,ngeri tkut main curang kaya 2019,negatif
4,lah sekarang udah keliatan curangnya,negatif
5,klo dipaksa satu putaran kecurangan tambah ric...,negatif


In [47]:
# Mengubah kata singkat menjadi normal
norm = {'sy':'saya','tau':'tahu','perubahancoblos':'perubahan coblos','utk':'untuk',
        'yg':'yang','org':'orang','pd':'pada','ganjar':'','mahfud':'','prabowo':'',
        'gibran':'','anis':'','muhaimin':'','trus':'terus','tkut':'takut'}

def normalisasi(str_text):
    for i in norm:
        str_text = str_text.replace(i, norm[i])
    return str_text

data['full_text'] = data['full_text'].apply(lambda x:normalisasi(x))
data

,full_text,sentimen
0,dukungan terus mengalir semakin kuat,positif
1,tetap dukung,positif
2,mendukung,positif
3,ngeri takut main curang kaya 2019,negatif
4,lah sekarang udah keliatan curangnya,negatif
5,klo dipaksa satu putaran kecurangan tambah ric...,negatif


In [48]:
# Memisahkan kata dalam suatu kalimat
tokenized = data['full_text'].apply(lambda x:x.split())
tokenized

0           [dukungan, terus, mengalir, semakin, kuat]
1                                      [tetap, dukung]
2                                          [mendukung]
3             [ngeri, takut, main, curang, kaya, 2019]
4           [lah, sekarang, udah, keliatan, curangnya]
5    [klo, dipaksa, satu, putaran, kecurangan, tamb...
Name: full_text, dtype: object

In [49]:
# Mengubah kata imbuhan menjadi kata dasar
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(text_cleaning):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in text_cleaning:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    print(d_clean)
    return d_clean

tokenized = tokenized.apply(stemming)

dukung terus alir makin kuat
tetap dukung
dukung
ngeri takut main curang kaya 2019
lah sekarang udah liat curang
klo paksa satu putar curang tambah ricuh suasana


In [50]:
data = pd.merge(tokenized, data[['sentimen']], left_index=True, right_index=True)

In [51]:
data.to_csv("hasil_stemming_coba.csv", index=False)

In [52]:
data = pd.read_csv("hasil_stemming_coba.csv")
data

,full_text,sentimen
0,dukung terus alir makin kuat,positif
1,tetap dukung,positif
2,dukung,positif
3,ngeri takut main curang kaya 2019,negatif
4,lah sekarang udah liat curang,negatif
5,klo paksa satu putar curang tambah ricuh suasana,negatif


In [53]:
# ekstaksi fitur tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer

In [59]:
x = data['full_text']
x

0                        dukung terus alir makin kuat
1                                        tetap dukung
2                                              dukung
3                   ngeri takut main curang kaya 2019
4                       lah sekarang udah liat curang
5    klo paksa satu putar curang tambah ricuh suasana
Name: full_text, dtype: object

In [60]:
tfidf_vectorizer = TfidfVectorizer()
x_tfidf = tfidf_vectorizer.fit_transform(x)

In [61]:
print(x_tfidf)

  (0, 6)	0.47249269128933835
  (0, 10)	0.47249269128933835
  (0, 1)	0.47249269128933835
  (0, 20)	0.47249269128933835
  (0, 3)	0.32711255969869474
  (1, 21)	0.8221903715494888
  (1, 3)	0.5692126078464125
  (2, 3)	1.0
  (3, 0)	0.42720625161336534
  (3, 4)	0.42720625161336534
  (3, 2)	0.2957601949422701
  (3, 9)	0.42720625161336534
  (3, 18)	0.42720625161336534
  (3, 11)	0.42720625161336534
  (4, 8)	0.47249269128933835
  (4, 22)	0.47249269128933835
  (4, 16)	0.47249269128933835
  (4, 7)	0.47249269128933835
  (4, 2)	0.32711255969869474
  (5, 17)	0.3656534077034961
  (5, 14)	0.3656534077034961
  (5, 19)	0.3656534077034961
  (5, 13)	0.3656534077034961
  (5, 15)	0.3656534077034961
  (5, 12)	0.3656534077034961
  (5, 5)	0.3656534077034961
  (5, 2)	0.25314639646604836


In [62]:
tfidf_vectorizer.get_feature_names()

['2019',
 'alir',
 'curang',
 'dukung',
 'kaya',
 'klo',
 'kuat',
 'lah',
 'liat',
 'main',
 'makin',
 'ngeri',
 'paksa',
 'putar',
 'ricuh',
 'satu',
 'sekarang',
 'suasana',
 'takut',
 'tambah',
 'terus',
 'tetap',
 'udah']